In [ ]:
from mec_env_var import *
from helper import *
import tensorflow as tf
import ipdb as pdb
import time

for k in range(10):
    
    tf.reset_default_graph()
    
    print('---------' + str(k) + '------------')
    
    MAX_EPISODE = 100
    MAX_EPISODE_LEN = 10000

    NUM_T = 1
    NUM_R = 4
    SIGMA2 = 1e-9

    t_factor = 0.2
    noise_sigma = 0.12

    config = {'state_dim':3, 'action_dim':2};
    train_config = {'minibatch_size':64, 'actor_lr':0.0001, 'tau':0.001, 
                    'critic_lr':0.001, 'gamma':0.99, 'buffer_size':250000, 
                    'random_seed':int(time.clock()*1000%1000), 'noise_sigma':noise_sigma, 'sigma2':SIGMA2}
    
#     rate = 3.0
    res_path = 't_M_02_nB/'
    model_path = 'model_M_02_nB/my_train_model_'+str(k)+'-2000'
    meta_path = model_path+'.meta'    
    user_config = [{'id':'1', 'model':'AR', 'num_r':NUM_R, 'rate':1.0, 'dis':100, 'action_bound':2, 
                    'data_buf_size':100, 't_factor':t_factor, 'penalty':1000},
                   {'id':'2', 'model':'AR', 'num_r':NUM_R, 'rate':2.0, 'dis':100, 'action_bound':2, 
                    'data_buf_size':100, 't_factor':t_factor, 'penalty':1000},
                   {'id':'3', 'model':'AR', 'num_r':NUM_R, 'rate':3.0, 'dis':100, 'action_bound':2, 
                    'data_buf_size':100, 't_factor':t_factor, 'penalty':1000}]
    
    print(user_config)
    # 0. initialize the session object
    sess = tf.Session() 

    # gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.2)
    # sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)) 

    # 1. include all user in the system according to the user_config
    user_list = [];
    for info in user_config:
        info.update(config)
        info['model_path'] = model_path
        info['meta_path'] = info['model_path']+'.meta'
        user_list.append(MecTermLD(sess, info, train_config))
        print('Initialization OK!----> user ' + info['id'])

    # 2. create the simulation env
    env = MecSvrEnv(user_list, NUM_R, SIGMA2, MAX_EPISODE_LEN)

#     sess.run(tf.global_variables_initializer())

#     env.init_target_network()

    # #Create a saver object which will save all the variables
    # saver = tf.train.Saver() 

    res_r = []
    res_p = []
    res_b = []
    res_o = []
    res_d = []
    # 3. start to explore for each episode
    for i in range(MAX_EPISODE):

        cur_init_ds_ep = env.reset(isTrain=False)
        
        cur_r_ep = np.zeros(len(user_list))
        cur_p_ep = np.zeros(len(user_list))
        cur_op_ep = np.zeros(len(user_list))
        cur_ts_ep = np.zeros(len(user_list))
        cur_ps_ep = np.zeros(len(user_list))
        cur_rs_ep = np.zeros(len(user_list))
        cur_ds_ep = np.zeros(len(user_list))
        cur_ch_ep = np.zeros(len(user_list))
        cur_of_ep = np.zeros(len(user_list))

        for j in range(MAX_EPISODE_LEN):

            # first try to transmit from current state
            [cur_r, done, cur_p, cur_op, temp, cur_ts, cur_ps, cur_rs, cur_ds, cur_ch, cur_of] = env.step_transmit()

            cur_r_ep += cur_r
            cur_p_ep += cur_p
            cur_op_ep += cur_op
            cur_ts_ep += cur_ts
            cur_ps_ep += cur_ps
            cur_rs_ep += cur_rs
            cur_ds_ep += cur_ds
            cur_ch_ep += cur_ch
            cur_of_ep += cur_of

            if done:
                res_r.append(cur_r_ep/MAX_EPISODE_LEN)
                res_p.append(cur_p_ep/MAX_EPISODE_LEN)
                res_b.append(cur_ds_ep/MAX_EPISODE_LEN)
                res_o.append(cur_of_ep/MAX_EPISODE_LEN)
                res_d.append(cur_ds)
                print('%d:r:%s,p:%s,op:%s,tr:%s,pr:%s,rev:%s,dbuf:%s,ch:%s,ibuf:%s,rbuf:%s' % (i, cur_r_ep/MAX_EPISODE_LEN, cur_p_ep/MAX_EPISODE_LEN, cur_op_ep/MAX_EPISODE_LEN, cur_ts_ep/MAX_EPISODE_LEN, cur_ps_ep/MAX_EPISODE_LEN, cur_rs_ep/MAX_EPISODE_LEN, cur_ds_ep/MAX_EPISODE_LEN, cur_ch_ep/MAX_EPISODE_LEN, cur_init_ds_ep, cur_ds))

    name = res_path+'test_1000_' + time.strftime("%b_%d_%Y_%H_%M_%S", time.localtime(time.time()))
    np.savez(name, res_r, res_p, res_b, res_o, res_d)
    
    sess.close()


---------0------------
[{'penalty': 1000, 'num_r': 4, 'rate': 1.0, 'action_bound': 2, 'data_buf_size': 100, 'model': 'AR', 'id': '1', 't_factor': 0.2, 'dis': 100}, {'penalty': 1000, 'num_r': 4, 'rate': 2.0, 'action_bound': 2, 'data_buf_size': 100, 'model': 'AR', 'id': '2', 't_factor': 0.2, 'dis': 100}, {'penalty': 1000, 'num_r': 4, 'rate': 3.0, 'action_bound': 2, 'data_buf_size': 100, 'model': 'AR', 'id': '3', 't_factor': 0.2, 'dis': 100}]
INFO:tensorflow:Restoring parameters from model_M_02_nB/my_train_model_0-2000
Initialization OK!----> user 1
INFO:tensorflow:Restoring parameters from model_M_02_nB/my_train_model_0-2000
Initialization OK!----> user 2
INFO:tensorflow:Restoring parameters from model_M_02_nB/my_train_model_0-2000
Initialization OK!----> user 3
0:r:[-1.82965702 -3.94752605 -8.3769811 ],p:[0.3635054  0.87111373 2.56827862],op:[0.09233615 0.05375821 0.05082056],tr:[0.01887723 0.78338568 2.12991285],pr:[1.38593584 1.3993614  1.25139185],rev:[1.0076 2.0108 2.9898],dbuf:[1.1

22:r:[-1.81470293 -3.89406405 -8.34828979],p:[0.35918691 0.85869697 2.56506723],op:[0.09274483 0.05616065 0.05245247],tr:[0.01635126 0.78907031 2.1298509 ],pr:[1.3827397  1.38232181 1.25722171],rev:[1.0003 1.9856 2.9949],dbuf:[1.13854931 2.580436   3.89156299],ch:[3.9496079  4.08194688 4.0161877 ],ibuf:[0, 0, 0],rbuf:[0.06520184 5.         2.        ]
23:r:[-1.77930104 -3.83403388 -8.35503707],p:[0.35245127 0.83883962 2.56848531],op:[0.09262704 0.05705739 0.05178148],tr:[0.01336523 0.81611002 2.11754914],pr:[1.37770552 1.36387906 1.24683297],rev:[0.9873 1.9951 2.9712],dbuf:[1.11143053 2.55279982 3.89312936],ch:[4.06206893 4.16528812 4.04209124],ibuf:[0, 0, 0],rbuf:[1. 1. 3.]
24:r:[-1.77454822 -3.8637354  -8.39289618],p:[0.34882253 0.85168876 2.58796712],op:[0.09480185 0.05392691 0.0509446 ],tr:[0.01472119 0.77685565 2.09449923],pr:[1.37486589 1.37979331 1.28021748],rev:[0.9816 1.9799 2.9861],dbuf:[1.10912431 2.56563008 3.89384094],ch:[4.17195028 4.07993615 3.93812352],ibuf:[0, 0, 0],rb

46:r:[-1.84268441 -3.8718964  -8.243358  ],p:[0.36815849 0.85003911 2.53493616],op:[0.09153845 0.05555624 0.05266237],tr:[0.01594453 0.79112982 2.16038965],pr:[1.39124467 1.37679286 1.2223639 ],rev:[1.0128 1.9848 2.9901],dbuf:[1.15411318 2.57588213 3.83520116],ch:[4.07373513 4.02743937 4.16584695],ibuf:[0, 0, 0],rbuf:[1.07031488 4.         2.1028935 ]
47:r:[-1.81105952 -3.92967338 -8.24254436],p:[0.36182288 0.86714917 2.5388258 ],op:[0.09056975 0.05506512 0.05301855],tr:[0.01297325 0.80019612 2.15789026],pr:[1.38409218 1.38461461 1.22390196],rev:[0.9996 2.005  2.9721],dbuf:[1.13284281 2.606556   3.82356956],ch:[4.00915729 3.91764072 4.06587193],ibuf:[0, 0, 0],rbuf:[1.         1.         3.04319513]
48:r:[-1.81346967 -3.91356375 -8.38776123],p:[0.35757225 0.86662219 2.58065725],op:[0.09384821 0.0531775  0.05069697],tr:[0.01600979 0.77171777 2.11012147],pr:[1.38243229 1.39345882 1.27433649],rev:[0.998  1.9916 3.0026],dbuf:[1.13828593 2.59245546 3.90631598],ch:[4.07268856 3.92055491 4.005

70:r:[-1.80502061 -3.87190978 -8.27978048],p:[0.35765483 0.84965836 2.56594995],op:[0.09171652 0.05549323 0.05330087],tr:[0.01671234 0.78827009 2.09996663],pr:[1.37937355 1.38109142 1.26178849],rev:[0.9962 1.9902 2.9691],dbuf:[1.1328474  2.57700826 3.80159855],ch:[3.92822218 4.04160589 3.94128279],ibuf:[0, 0, 0],rbuf:[0.         2.71532302 4.        ]
71:r:[-1.82794406 -3.93728066 -8.48089199],p:[0.36344765 0.87001432 2.6041985 ],op:[0.09216643 0.05505305 0.05147721],tr:[0.01879711 0.7842845  2.08306802],pr:[1.38253339 1.39632074 1.3008256 ],rev:[1.0023 2.0006 3.0026],dbuf:[1.14589489 2.60893242 3.96192571],ch:[4.0194587  3.95836106 3.86794675],ibuf:[0, 0, 0],rbuf:[0.07423462 2.         5.06832802]
72:r:[-1.82137147 -3.96803902 -8.54172717],p:[0.36169063 0.88019193 2.59891872],op:[0.09225965 0.05225699 0.04811928],tr:[0.01567953 0.7665906  2.11921536],pr:[1.38458448 1.40625564 1.2834551 ],rev:[1.0021 2.0046 3.0237],dbuf:[1.14183864 2.62892648 4.05956394],ch:[3.96375577 3.83915085 4.049

94:r:[-1.77595519 -3.93330427 -8.39921102],p:[0.34997843 0.8691777  2.58728283],op:[0.09235092 0.05453812 0.05719636],tr:[0.01667555 0.81066385 2.05996647],pr:[1.37112749 1.38504701 1.30145373],rev:[0.9842 2.0173 2.982 ],dbuf:[1.11412062 2.60734077 3.88781582],ch:[3.857556   4.09520686 3.8183548 ],ibuf:[0, 0, 0],rbuf:[0. 1. 3.]
95:r:[-1.8111892  -3.90749214 -8.39269916],p:[0.35725799 0.86157111 2.56821378],op:[0.0926551  0.05514977 0.05164398],tr:[0.01708477 0.78466731 2.13619239],pr:[1.37813428 1.38538164 1.25776447],rev:[0.9959 1.9913 3.0084],dbuf:[1.13920376 2.59256296 3.94122955],ch:[3.93378791 3.94497187 4.06450385],ibuf:[0, 0, 0],rbuf:[1.0867005 3.        1.       ]
96:r:[-1.83327385 -3.90825896 -8.32593602],p:[0.36712604 0.86529002 2.55933806],op:[0.09024535 0.05230571 0.04850375],tr:[0.01698707 0.75291732 2.14173145],pr:[1.386473   1.39484604 1.23883417],rev:[1.0075 1.9762 2.9955],dbuf:[1.14816383 2.59133437 3.8878155 ],ch:[4.01155725 3.88435639 4.17244016],ibuf:[0, 0, 0],rbuf:

...

34:r:[ -1.75992129  -6.11851564 -10.8978821 ],p:[0.33528536 1.2033362  2.76053936],op:[0.07919289 0.90807749 1.23946063],tr:[0.01819586 0.23387591 2.11041024],pr:[1.26987959 2.5198421  2.5198421 ],rev:[0.9952 2.0035 3.0223],dbuf:[1.16370597 2.36961032 3.62235265],ch:[4.04292391 4.07191518 4.0616746 ],ibuf:[0, 0, 0],rbuf:[1. 3. 3.]
35:r:[ -1.72648159  -6.14268304 -10.76630397],p:[0.32820955 1.22578094 2.72453894],op:[0.07844515 0.88578823 1.27546105],tr:[0.01338307 0.2330468  2.1239432 ],pr:[1.26135611 2.5198421  2.5198421 ],rev:[0.9799 2.0263 2.9709],dbuf:[1.14146522 2.39943087 3.45787999],ch:[3.90143909 4.08806122 4.08654925],ibuf:[0, 0, 0],rbuf:[1.10704131 2.         4.        ]
36:r:[ -1.78515335  -6.10847629 -10.80494778],p:[0.34180695 1.20320688 2.7454893 ],op:[0.07817309 0.90687814 1.25451069],tr:[0.01547234 0.2296905  2.10334088],pr:[1.27927992 2.5198421  2.5198421 ],rev:[1.0048 2.0026 3.0002],dbuf:[1.18149158 2.36038281 3.50618474],ch:[4.00686359 4.11143445 3.95994769],ibuf:[0,

58:r:[ -1.76916007  -6.12122807 -10.82955717],p:[0.33646743 1.21197114 2.7440093 ],op:[0.0792539  0.89688128 1.25599069],tr:[0.01626753 0.22783171 2.13662065],pr:[1.27502658 2.5198421  2.5198421 ],rev:[1.0038 2.0098 3.0178],dbuf:[1.17214677 2.37940403 3.5369465 ],ch:[4.01009699 4.00628055 4.10036803],ibuf:[0, 0, 0],rbuf:[0.         4.65504411 3.        ]
59:r:[ -1.81793896  -6.08960496 -10.80790506],p:[0.35150865 1.19901115 2.72386358],op:[0.0770758  0.90602816 1.2761364 ],tr:[0.01743631 0.22459695 2.15020286],pr:[1.28487846 2.5198421  2.51984209],rev:[1.0202 2.0001 2.9925],dbuf:[1.20096257 2.34940794 3.50988138],ch:[3.94301122 4.05350974 4.06967053],ibuf:[0, 0, 0],rbuf:[3.         6.99831381 6.7574431 ]
60:r:[ -1.78095499  -6.10092966 -10.83251413],p:[0.34054142 1.20214223 2.74424246],op:[0.07818546 0.90697128 1.25575752],tr:[0.02034682 0.22623162 2.09953909],pr:[1.27609374 2.5198421  2.51984209],rev:[1.0078 2.0046 2.9988],dbuf:[1.17937652 2.3533783  3.54064272],ch:[4.15056293 4.02011

82:r:[ -1.76563541  -6.03278896 -10.95902029],p:[0.33718871 1.16517849 2.78804035],op:[0.07892002 0.93661226 1.21195964],tr:[0.01888447 0.2172965  2.10261356],pr:[1.2710876 2.5198421 2.5198421],rev:[0.9959 1.9656 3.0658],dbuf:[1.16677244 2.28650933 3.69877539],ch:[4.04084897 4.13999405 3.9420409 ],ibuf:[0, 0, 0],rbuf:[1. 0. 2.]
83:r:[ -1.77168354  -6.12626349 -10.84143764],p:[0.33841592 1.20661456 2.74733883],op:[0.07727312 0.9050666  1.25266117],tr:[0.01816267 0.23017917 2.12400438],pr:[1.269671  2.5198421 2.5198421],rev:[1.0038 1.9976 2.9988],dbuf:[1.1753818  2.37862649 3.55179707],ch:[4.23397679 4.00000553 4.02244285],ibuf:[0, 0, 0],rbuf:[1.31302819 1.         2.        ]
84:r:[ -1.73799256  -6.09445615 -10.86301131],p:[0.32928801 1.19331976 2.76131593],op:[0.07971681 0.91471172 1.23868404],tr:[0.01264889 0.22108936 2.08477121],pr:[1.27361692 2.5198421  2.51984209],rev:[0.9887 1.9834 2.9993],dbuf:[1.14997867 2.34799149 3.57876419],ch:[3.96383414 3.89562159 3.93742603],ibuf:[0, 0, 0]

4:r:[-1.51510549 -6.66896863 -7.61609102],p:[0.26834422 1.36920071 2.02539418],op:[0.02216374 0.63079929 0.02230709],tr:[2.69035144e-01 3.89289038e-14 1.27467642e+00],pr:[0.97727637 2.5198421  1.76685978],rev:[1.0109 1.9995 3.0088],dbuf:[1.16761196 3.33621079 4.4008606 ],ch:[4.01124445 4.00449566 4.07928983],ibuf:[0, 0, 0],rbuf:[1. 1. 3.]
5:r:[-1.46553386 -6.6891989  -7.57793218],p:[0.2573464  1.37109258 2.00968803],op:[0.02313401 0.62890742 0.02155124],tr:[2.54333452e-01 1.59915275e-13 1.26827660e+00],pr:[0.97151841 2.5198421  1.74964091],rev:[0.9807 2.0017 2.9856],dbuf:[1.13071629 3.36149863 4.39431705],ch:[4.02597463 3.99437815 4.13390258],ibuf:[0, 0, 0],rbuf:[1.09229109 5.2817369  6.94193686]
6:r:[-1.47731993 -6.65509193 -7.53823406],p:[0.25898195 1.36213069 2.01435919],op:[0.02260976 0.63786931 0.02183385],tr:[2.51662484e-01 3.25130299e-13 1.25276787e+00],pr:[0.97544408 2.5198421  1.76489328],rev:[0.9901 1.9988 2.9849],dbuf:[1.14267063 3.31886491 4.33230997],ch:[3.90286831 4.04389

27:r:[-1.50212084 -6.63476131 -7.85503533],p:[0.26515722 1.34197801 2.08658428],op:[0.02248576 0.65802199 0.0210713 ],tr:[2.63959152e-01 2.02731697e-13 1.25892630e+00],pr:[0.97604959 2.5198421  1.80543828],rev:[1.0034 1.9762 3.0324],dbuf:[1.15854359 3.29345163 4.5496552 ],ch:[4.05494376 4.04834138 3.99066025],ibuf:[0, 0, 0],rbuf:[1.16209636 7.2023685  1.56349918]
28:r:[-1.5006734  -6.6638851  -7.60830809],p:[0.26540275 1.35980269 2.02816112],op:[0.02287602 0.64019731 0.02009451],tr:[2.69973173e-01 2.23783495e-13 1.24597411e+00],pr:[0.97377946 2.5198421  1.768509  ],rev:[1.002  1.994  2.9843],dbuf:[1.1551448  3.32985637 4.38974602],ch:[4.06521008 4.04279359 4.09371408],ibuf:[0, 0, 0],rbuf:[0.0706315  3.3214211  7.34674339]
29:r:[-1.52672927 -6.81424952 -7.77458489],p:[0.27181957 1.39431558 2.07431079],op:[0.02225913 0.60568442 0.01928088],tr:[2.65827723e-01 1.20754347e-13 1.22649218e+00],pr:[0.97917738 2.5198421  1.80554489],rev:[1.0089 2.0222 3.0027],dbuf:[1.17321486 3.51781191 4.48425

In [2]:
name = res_path+'/test_1000_' + time.strftime("%b_%d_%Y_%H_%M_%S", time.localtime(time.time()))
# print(name)
np.savez(name, res_r, res_p, res_b, res_o, res_d)

In [ ]:
graph = tf.get_default_graph()
# variable_names = [v.name for v in tf.trainable_variables()]
# values = sess.run(variable_names)
# tf.trainable_variables()

inputs = graph.get_tensor_by_name("input_1/X"+":0")
scaled_out = graph.get_tensor_by_name("output_1"+":0")
print sess.run(scaled_out, feed_dict={inputs: [[7.22254768,3.9729721,0.7729721]]})
#Create a saver object which will save all the variables
saver = tf.train.Saver()
 
#Now, save the graph
saver.save(sess, 'model/my_train_model',global_step=2000)

# user_list[0].agent.actor.predict([[7.22254768,3.9729721,0.7729721]]) # [[0.63916105, 0.20230901]]